In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
import scipy.integrate as spi
import pylab as pl
import matplotlib as mlb
from lmfit import minimize, Parameters, Parameter, report_fit
from scipy.signal import savgol_filter
import operator
from pyDOE import *

In [2]:
data2=pd.read_csv("data2.csv")
alive1911=pd.read_csv("alivein1911.csv")
cause=pd.Series(data2['Cause of Death'])
measles=data2[cause.str.contains('measles')==True]
cause=pd.Series(alive1911['Cause of Death'])
measles1911=alive1911[cause.str.contains('measles')==True]

In [3]:
measles_only=np.array([])
for i in range(len(measles1911)):
    
    if measles1911['Cause of Death'].values[i]=='measles':
        measles_only=np.hstack((measles_only,measles1911['Deathdate'].values[i]))

In [4]:
measles_diarrhoea=np.array([])
for i in range(len(measles1911)):
    
    if measles1911['Cause of Death'].values[i]=='measles, diarrhoea':
        measles_diarrhoea=np.hstack((measles_diarrhoea,measles1911['Deathdate'].values[i]))
    
    elif measles1911['Cause of Death'].values[i]=='measles, diahrroea':
        measles_diarrhoea=np.hstack((measles_diarrhoea,measles1911['Deathdate'].values[i]))  
    elif measles1911['Cause of Death'].values[i]=='measles, ileo-colitis':
        measles_diarrhoea=np.hstack((measles_diarrhoea,measles1911['Deathdate'].values[i])) 
    elif measles1911['Cause of Death'].values[i]=='measles, diarrhoea,\x0bheart failure':
        measles_diarrhoea=np.hstack((measles_diarrhoea,measles1911['Deathdate'].values[i])) 
        
    elif measles1911['Cause of Death'].values[i]=='measles, diarrhoea,\x0bchildbirth':
        measles_diarrhoea=np.hstack((measles_diarrhoea,measles1911['Deathdate'].values[i])) 
        
    elif measles1911['Cause of Death'].values[i]== 'measles, diarrhoea,\x0bmiscarriage':
        measles_diarrhoea=np.hstack((measles_diarrhoea,measles1911['Deathdate'].values[i])) 
    
    elif measles1911['Cause of Death'].values[i]== 'measles, gastritis':
        measles_diarrhoea=np.hstack((measles_diarrhoea,measles1911['Deathdate'].values[i])) 
        

In [5]:
measles_only=sorted(measles_only,key=lambda x: datetime.strptime(x, '%d/%m/%Y'))
measles_diar=sorted(measles_diarrhoea,key=lambda x: datetime.strptime(x, '%d/%m/%Y'))
deathmeasles=pd.unique(measles_diar)
daterange=pd.date_range(deathmeasles[0],deathmeasles[-1],freq='D')

In [6]:
values3=np.zeros(len(daterange))
deathtime3=np.zeros(len(daterange))
for i in range(len(daterange)):
    for j in range(len(measles_only)):
        if daterange[i]==datetime.strptime(measles_only[j],'%d/%m/%Y'):
            #print(i)
            #print(j)
            values3[i]=values3[i]+1

In [7]:
values2=np.zeros(len(daterange))
deathtime2=np.zeros(len(daterange))
for i in range(len(daterange)):
    for j in range(len(measles_diarrhoea)):
        if daterange[i]==datetime.strptime(measles_diarrhoea[j],'%d/%m/%Y'):
            #print(i)
            #print(j)
            values2[i]=values2[i]+1

In [8]:
measles_timeseries=np.hstack((np.zeros(30),values3,np.zeros(10)))
coinf_timeseries=np.hstack((np.zeros(30),values2,np.zeros(10)))
measles_timeseries2=np.hstack((np.zeros(10),values3,np.zeros(30)))
coinf_timeseries2=np.hstack((np.zeros(10),values2,np.zeros(30)))
measles_timeseries3=np.hstack((np.zeros(1),values3,np.zeros(39)))
coinf_timeseries3=np.hstack((np.zeros(1),values2,np.zeros(39)))

In [9]:
def seir_model(INP,t,ps):
    try:
        
        beta1=ps['beta1'].value
#     beta=np.hstack((ps['beta0'].value,ps['beta1'].value))
        
        gamma1=ps['gamma1'].value
        d=ps['d'].value
        
        sigma=ps['sigma'].value
        
        
    except:
        beta1,gamma1,d,sigma=ps
    N=2618
    
    Y=np.zeros((5))
    
    V = INP   
    
    #susceptible
    Y[0]=-(beta1/N)*V[0]*V[2]
    
    #exposed
    Y[1]=(beta1/N)*V[0]*V[2]-sigma*V[1]
    
    #infected
    Y[2]=sigma*V[1]-d*V[2]-gamma1*V[2]
    
    #recovered
    Y[3]=gamma1*V[2]
    
    #dead
    Y[4]=d*V[2]
    
    return Y
    
    
    

In [10]:
N=2618
INPUT=(N-2,2,0,0,0)

ts_len=len(measles_timeseries)

exp_period=0

def residual(ps,INP, ts, data):

    RES = spi.odeint(seir_model,INP,ts,args=(ps,))
    res_measles=RES[1:ts_len+1,4]-RES[0:ts_len,4]

    model=res_measles[exp_period:ts_len]

    return (model - data).ravel()


TS=1.0
ND=float(ts_len+1)
t_start = 0.0; t_end = ND; t_inc = TS
t_range = np.arange(t_start, t_end+t_inc, t_inc)

measles_ts_smooth = savgol_filter(measles_timeseries, 41, 3)
coinf_ts_smooth = savgol_filter(coinf_timeseries, 41, 3)
#time_series=np.array((measles_ts_smooth,coinf_ts_smooth))
#time_series=np.array((measles_timeseries,coinf_timeseries))
time_series1=measles_timeseries+coinf_timeseries

time_series1=time_series1[exp_period:ts_len]
#time_series=np.array((measles_timeseries,coinf_timeseries,71,88))
# t = np.linspace(0, 10, 10)
# data = g(t, x0, true_params)
# data += np.random.normal(size=data.shape)

# beta1     0.4145     
# c1        0.0196  


# omega0    0.1271   
# sigma     0.1428 


# set parameters incluing bounds
params = Parameters()

params.add('beta1',value=0.41,min=0.03,max=3)

params.add('gamma1',value=0.12,min=1/21,max=1/4)
params.add('d',value=0.019,min=0.003,max=0.1)

params.add('sigma',value=0.14,min=1/21,max=1/3)


# fit model and find predicted values
result = minimize(residual, params, args=(INPUT,t_range, time_series1), method='leastsq')
final = time_series1 + result.residual.reshape(time_series1.shape)

In [12]:
report_fit(result)

[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 34
    # data points      = 160
    # variables        = 4
    chi-square         = 350.347709
    reduced chi-square = 2.24581865
    Akaike info crit   = 133.400368
    Bayesian info crit = 145.701063
[[Variables]]
    beta1:   0.40890039 +/- 872.447768 (213364.38%) (init = 0.41)
    gamma1:  0.12664773 +/- 270.235005 (213375.32%) (init = 0.12)
    d:       0.01960123 +/- 41.8230919 (213369.72%) (init = 0.019)
    sigma:   0.14624539 +/- 312.078529 (213393.75%) (init = 0.14)
[[Correlations]] (unreported correlations are < 0.100)
    C(gamma1, d)     =  1.000
    C(beta1, sigma)  = -1.000
    C(beta1, gamma1) =  1.000
    C(beta1, d)      =  1.000
    C(gamma1, sigma) = -1.000
    C(d, sigma)      = -1.000


In [13]:
result.params.pretty_print()

Name       Value      Min      Max   Stderr     Vary     Expr Brute_Step
beta1     0.4089     0.03        3    872.4     True     None     None
d         0.0196    0.003      0.1    41.82     True     None     None
gamma1    0.1266  0.04762     0.25    270.2     True     None     None
sigma     0.1462  0.04762   0.3333    312.1     True     None     None


In [12]:
%matplotlib qt
plt.figure(figsize=(15,10))
#RES = spi.odeint(coinf_exp_model,INPUT,t_range,result.params())
#plt.plot(RES[:,5])
plt.rcParams.update({'font.size': 22})
plt.plot(time_series1,label="Data")
ts_smooth=savgol_filter(time_series1, 51, 3)
#plt.plot(ts_smooth,label="smoothed data")
plt.plot(final,label="Least squares fit")
#plt.plot(final[1],label="fit to coinfection")
measles_ts_smooth = savgol_filter(measles_timeseries, 41, 3)
coinf_ts_smooth = savgol_filter(coinf_timeseries, 41, 3)
#plt.plot(measles_ts_smooth)

# beta0,beta1,gamma0,gamma1,c=1.2,1.5,0.1,0.1,0.3
# true_params = np.array((beta0,beta1,gamma0,gamma1,c))
# data = g(t_range, INPUT, true_params)
#data += np.random.normal(size=data.shape)
# plt.plot(data[0],label="Measles")
# plt.plot(data[1],label="coinf")
plt.legend()
plt.xlabel("Days")
plt.ylabel("Deaths")
plt.show()

ModuleNotFoundError: No module named 'PyQt4'